# Getting the Correct GCN Layers

Both the pygcn and gvae repos have implementations of the GCN layer, I want to see if they are both the same and how they work.

Update: I think they are both the same. 

Let's implement this one step at a time and understand the math along the way. 

## THE MATH: Encoder

They use a 2-layer GCN. Together the layers are doing probabilistic inference (in the encoder, specifically?): 

$$q(\mathbf{Z} | \mathbf{X, A}) = \Pi_{i = 1}^{N} q(\mathbf{z_i} | \mathbf{X, A})$$

Where we define the conditional probability of the stochastic latent variable $\mathbf{z_i}$ as: 

$$q(\mathbf{z_i} | \mathbf{X, A}) = \mathcal{N}(\mathbf{z_i} | \mathbf{\mu_i}, \text{diag}(\mathbf{\sigma_i}^2))$$

Therefore, the stochastic latent variable is from a normal distribution that is parameterized by $\mathbf{\mu_i}$ and $\text{diag}(\mathbf{\sigma_i})$. We can write these explicitly as: 

The mean is a matrix of mean vectors, $\mathbf{\mu_i}$, i.e.: $\mathbf{\mu} = \text{GCN}_{\mathbf{\mu}} (\mathbf{X, A})$   
And the log of the standard deviation is: $\log(\mathbf{\sigma}) = \text{GCN}_{\mathbf{\sigma}} (\mathbf{X, A})$

From this we can see that the neural network we are building is learning to represent an appropriate normal distribution $\mathcal{N}(\mu, \sigma)$ that represents the distribution of stochastic latent variables that are embedded equivalents of the graph defined by $\mathbf{X, A}$. NOTE: If we instead want to implement a deterministic AE, then we can still use the GCN layers, but we simply take the mean of the layers' output and use the log likelihood instead of the KL divergence as our loss function. We can define the neural network, i.e. the GCN layer itself, as follows: 

$$\text{GCN}(\mathbf{X, A}) = \mathbf{\tilde{A}} \text{ReLU} (\mathbf{\tilde{A}X W_0}) \mathbf{W_1}$$

where the weight matrices are $\mathbf{W_0, W_1}$. If this feels like it is coming out of left field, my blog post here explains the derivation of this expression in detail: https://sassafras13.github.io/graphConv/. 

Both the mean and standard deviation computed by the GCN (i.e. $\text{GCN}_{\mu}$ and $\text{GCN}_{\sigma}$) share the same first layer parameters/weights $\mathbf{W_0}$. 

Recall that we can define $\text{ReLU}(\cdot) = \max(0, \cdot)$. And $\mathbf{\tilde{A}}$ is the normalized adjacency matrix we computed previously. 

### Matrix Dimensions

To better understand the code below, I want to explicitly write out the matrix dimensions. 

- [ ] N is number of nodes in the graph
- [ ] F is number of features for each node in the graph 
- [ ] Nhid is the number of hidden features between layer 1 and 2 of the encoder  
- [ ] Nclass is the number of classes that the GCN is trying to predict - this is the size of the output. For us I think this would be Nlatent, i.e. how big we want our latent space to be. 

- [ ] $\mathbf{\tilde{A}} = N \times N$
- [ ] $\mathbf{X} = N \times F$   
- [ ] $\mathbf{W_0} = F \times Nhid$  
- [ ] $\mathbf{W_1} = Nhid \times Nlatent$
- [ ] $\mathbf{Z} = N \times Nlatent$

So all together: 

$$\begin{aligned} \mathbf{\tilde{A}} \text{ReLU} (\mathbf{\tilde{A}X W_0}) \mathbf{W_1} &= (N \times N) \times \bigg( (N \times N) \times (N \times F) \times (F \times \text{Nhid}) \bigg) \times (\text{Nhid} \times \text{Nlatent}) \\ 
&= (N \times N) \times \bigg( (N \times \text{Nhid}) \bigg) \times (\text{Nhid} \times \text{Nlatent}) \\
&= (N  \times \text{Nlatent}) \\
&= \mathbf{Z} \end{aligned}$$

Now that we have understood this part, let us go to the code to see how this GCN is implemented.

In [4]:
import math
import torch
from torch.nn.parameter import Parameter # this is a kind of tensor that is automatically considered as the parameter of a class/module
from torch.nn.modules.module import Module # torch base class for all NN modules, always inherit from here


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        # according to: https://discuss.pytorch.org/t/super-model-in-init/97426
        # The super call delegates the function call to the parent class, 
        # which is nn.Module in your case. 
        # This is needed to initialize the nn.Module properly. 
        # from: https://realpython.com/python-super/
        # high level super() gives you access to methods in a superclass 
        # from the subclass that inherits from it
        # and from: https://medium.com/@ariellemesser/pytorch-nn-module-super-classes-sub-classes-inheritance-and-call-speci-3cc277407ff5
        # In the super class, nn.Module, there is a __call__ method which 
        # obtains the forward function from the subclass and calls it.
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features)) # why wouldn't you have bias?
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    # we define the reset_parameters method
    def reset_parameters(self):
        # why are we using the size of the weight matrix to compute std dev? 
        # self.weight.size(1) = Nhid for W0, Nlatent for W1
        # this expression is essentially assuming that the squared residual between the data and 
        # the mean is 1, and the number of data points in the sample is Nhid or Nlatent
        stdv = 1. / math.sqrt(self.weight.size(1)) 
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        # together, support and output are computing
        # A_tilde * X * W by doing
        # support = X * W
        # output = A_tilde * support = A_tilde * X * W
        support = torch.mm(input, self.weight) # mat mul
        output = torch.spmm(adj, support) # sparse mat mul
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

## Compare to VGAE Repo

In comparing this definition of the GCN layer to the VGAE repo, we have pretty much the same thing: 

In [9]:
import torch
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.metrics import roc_auc_score, average_precision_score
import scipy.sparse as sp
import numpy as np
import os
import time
import torch.nn as nn

In [10]:
class GraphConvSparse(nn.Module):
	def __init__(self, input_dim, output_dim, adj, activation = F.relu, **kwargs):
		super(GraphConvSparse, self).__init__(**kwargs)
		self.weight = glorot_init(input_dim, output_dim) 
		self.adj = adj
		self.activation = activation

	def forward(self, inputs):
		x = inputs
		x = torch.mm(x,self.weight)
		x = torch.mm(self.adj, x)
		outputs = self.activation(x)
		return outputs

The primary differences here are that:
- This implementation incorporates the activation function in the layer definition (the implementation above does it at the model level)  
- This implementation initializes the weights using the function glorot_init which is given below.

In [11]:
def glorot_init(input_dim, output_dim):
	init_range = np.sqrt(6.0/(input_dim + output_dim))
	initial = torch.rand(input_dim, output_dim)*2*init_range - init_range
	return nn.Parameter(initial)

Ref: https://jamesmccaffrey.wordpress.com/2017/06/21/neural-network-glorot-initialization/

The Glorot initialization method (also referred to as Xavier initialization) was proposed by Glorot and Bengio in 2010 (https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf). It is optimized for deep NNs that use ReLU activation. In normal weight activation, we set the weights and biases to uniform random values in the range [-0.01, +0.01]. But this does not work as well for deep NNs with ReLU activation. In this case, we use Glorot initialization which intializes each weight with a small Gaussian (or uniform) value with mean 0.0 and variance determined by the fan-in and fan-out of the weights. 

Fan-in and fan-out refer to the number of input nodes and output nodes/hidden nodes that the weights map to. If you are using the Gaussian distribution with Glorot, then you would compute: 

$$\sigma = \sqrt{\frac{2}{\text{fan-in} + \text{fan-out}}}$$
$$\text{weight} = \mathcal{N}(\mu = 0.0, \sigma)$$

If instead you were doing Glorot initialization with Uniform distribution (as we are above) then you would compute: 

$$\sigma = \sqrt{\frac{6}{\text{fan-in} + \text{fan-out}}}$$
$$\text{weight} = \mathcal{N}(\mu = 0.0, \sigma)$$